## This python notebook has examples of the Set-Coverage Problem and Partial Set-Coverage Problem
### load libraries

In [69]:
import geopandas as gpd
import pandas as pd
import os, sys, time
sys.path.append(r'/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets')
import GOSTnet as gn
import importlib
import networkx as nx
import osmnx as ox
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely.geometry import LineString, MultiLineString, Point

### read back in processed graph

In [113]:
G = nx.read_gpickle('./sampleData/nouakchott/biggest_subg.pickle')

In [74]:
gn.example_edge(G)

(0, 23857, {'Wkt': 'LINESTRING (-15.8975893 18.0394637, -15.8976921 18.039127)', 'id': 45978, 'infra_type': 'residential', 'osm_id': '667426151', 'key': 'edge_45978', 'length': 38.82328422750167, 'Type': 'legitimate', 'time': 13.976382321900601, 'mode': 'drive'})


## load origins and destinations

In [75]:
origins = pd.read_csv('./sampleData/nouakchott/origins_test1.csv')
origins['geometry'] = list(zip(origins['Lon'],origins['Lat']))
origins['geometry'] = origins['geometry'].apply(Point)
origins_gdf = gpd.GeoDataFrame(origins, crs = {'init':'epsg:4326'}, geometry = 'geometry')
origins_gdf = gn.pandana_snap(G, origins_gdf, target_crs = 'epsg:32628', add_dist_to_node_col = True)
origins = list(origins_gdf.NN)
origins = list(set(origins))

In [76]:
destinations = pd.read_csv('./sampleData/nouakchott/destinations_test1.csv')
destinations['geometry'] = list(zip(destinations['Lon'],destinations['Lat']))
destinations['geometry'] = destinations['geometry'].apply(Point)
destinations_gdf = gpd.GeoDataFrame(destinations, crs = {'init':'epsg:4326'}, geometry = 'geometry')
destinations_gdf = gn.pandana_snap(G, destinations_gdf, target_crs = 'epsg:32628', add_dist_to_node_col = True)
#destinations_gdf.NN is the nearest node of the road network
destinations = list(destinations_gdf.NN)
destinations = list(set(destinations))

## calculate OD Matrix

In [77]:
%time OD = gn.calculate_OD(G, origins, destinations, fail_value = 9999999999999)

print origins type
<class 'list'>
weighted_origins equals false
CPU times: user 11.7 s, sys: 0 ns, total: 11.7 s
Wall time: 12.9 s


In [78]:
OD_df = pd.DataFrame(OD, columns = destinations, index = origins)

In [107]:
import importlib
importlib.reload(gn)

peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.3 
osmnx version: 0.9 


<module 'GOSTnet' from '/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets/GOSTnet.py'>

## Set-Coverage Problem
### Objective: Determine the minimum number of facilities and their locations in order to cover all demands within a pre-specified maximum distance (or time) coverage

### inputs include a pre-specified maximum distance coverage of 3000 seconds

In [108]:
set_coverage_result = gn.optimize_set_coverage(OD_df,max_coverage = 3000)

number of origins
50
print totalCoveredFacilities
50
print percent coverage
1.0


In [109]:
set_coverage_result

[2440, 25486, 4965]

## Partial Set-Coverage Problem
### Objective: Determine the minimum number of facilities and their locations in order to cover a given fraction of the population within a pre-specified maximum distance (or time) coverage

### we need to produce a series that has each origin and its respective population

In [110]:
origins_w_demands_series = pd.Series(origins_gdf.demand.values,index=origins_gdf.NN)

### inputs include covering 90 percent of the population, a pre-specified maximum distance coverage of 2000 seconds, and a  series of origins with their population

In [115]:
partial_set_coverage_result = gn.optimize_partial_set_coverage(OD_df, pop_coverage = .9, max_coverage = 2000, origins_pop_series = origins_w_demands_series, existing_facilities = None)

print min_coverage
12419780.4


In [112]:
partial_set_coverage_result

[15739, 1646, 2440, 25486, 33112, 4092, 4965, 6724, 7011, 8994]